22. Calcular la longitud de arco de la curva:

$$ 8x = y^4 + \frac{2}{y^2} $$

Desde *y=1* hasta *y=2* empleando la integración de Romberg mediante R<sub>66</sub>. Además obtenga el valor exacto y el error. Emplee 15 decimales.

### Solución

Primero despejo *x* de la ecuación:

$$ x = \frac{y^4}{8} + \frac{1}{4y^2} $$

Por tanto la longitund de arco vendría dada por:

$$ L = \int_{1}^{2} \sqrt{1 + (x')^2} dx $$



In [1]:
# Desarrollo en Python

from sympy import *
from numpy import zeros, float64

# Voy a ocupar x como variable en vez de y

x = symbols("x")

# Mi función original
g= (x**4 / 8 ) + (1 / (4 * x**2))

# La función de la longitud de arco
f = sqrt(1 + (diff(g))**2)

# Aqui defino n con el R_nn dado, a y b
n = 6
a = 1
b = 2

# declaro mi matriz (n, n)
R = zeros((n, n), dtype=float64)


# Utilizo la función que he desarrollado para la regla compuesta del trapecio
def regla_compuesta_del_trapecio(f, a, b, n):
    # Defino mi h dentro de la función
    h = (b-a) / n
    # La suma interna es el factor de la fórmula, la inicio con f(x_0)
    suma_interna = f.subs(x, a)
    # Aquí hago la sumatoria desde f(x_1) hasta f(x_(n-1)) 
    for i in range(1, n):
        suma_interna += 2 * f.subs(x, a + i*h)
    # Sumo f(x_n)
    suma_interna += f.subs(x, b)
    return (h/2) * suma_interna

# Y encuentro cada valor con mi función donde "n" de la regla compuesta
# Se duplica en cada iteración

R11 = float(regla_compuesta_del_trapecio(f, a, b, 1))
R21 = float(regla_compuesta_del_trapecio(f, a, b, 2))
R31 = float(regla_compuesta_del_trapecio(f, a, b, 4))
R41 = float(regla_compuesta_del_trapecio(f, a, b, 8))
R51 = float(regla_compuesta_del_trapecio(f, a, b, 16))
R61 = float(regla_compuesta_del_trapecio(f, a, b, 32))

# Ahora estos valor les pongo el la primera columna de mi matriz R

R[0][0] = R11
R[1][0] = R21
R[2][0] = R31
R[3][0] = R41
R[4][0] = R51
R[5][0] = R61

# Saco mi valor exacto a modo de verificación, lo imprimo y la matriz
# Para verificar que todo parezca adecuado

valor_exacto = float(integrate(f, (x, a, b)))

print(valor_exacto)
print(R)




2.0625
[[2.53125    0.         0.         0.         0.         0.        ]
 [2.18344907 0.         0.         0.         0.         0.        ]
 [2.09311065 0.         0.         0.         0.         0.        ]
 [2.07018057 0.         0.         0.         0.         0.        ]
 [2.06442198 0.         0.         0.         0.         0.        ]
 [2.06298061 0.         0.         0.         0.         0.        ]]


In [9]:
# Ahora lleno el resto de la matriz con la versión espejo de la
# Interpolación de Richarson

for j in range(1, n):
    for i in range(j, n):
        # La j es diferente a la fórmula por la forma de indexar de Python
        R[i][j] = (4**(j) * R[i][j-1] - R[i-1][j-1]) / (4**(j) - 1)

#imprimo de forma que se vea bien la matriz
for i in R:
    for j in i:
        print(str(j).ljust(18), end="  ")
    print("\n")

# el valor R[n-1][n-1] es el valor aproximado

valor_aproximado = R[n-1][n-1]
error = abs(valor_aproximado - valor_exacto)
error_relativo = round(float(error/valor_exacto * 100), 10)

print("El valor exacto es:", valor_exacto)
print("El valor aproximado es:", round(valor_aproximado, 15))
print("El error es:", error)
print(f"El error relativo es: {error_relativo} %")

2.53125             0.0                 0.0                 0.0                 0.0                 0.0                 

2.183449074074074   2.0675154320987654  0.0                 0.0                 0.0                 0.0                 

2.093110652197387   2.062997844905158   2.062696672425584   0.0                 0.0                 0.0                 

2.070180570148941   2.0625372094661256  2.062506500436857   2.0625034818338612  0.0                 0.0                 

2.064421984029672   2.0625024553232487  2.0625001383803903  2.0625000373953672  2.062500023887765   0.0                 

2.0629806127785058  2.0625001556947837  2.0625000023862197  2.062500000227582   2.062500000081826   2.0625000000585554  

El valor exacto es: 2.0625
El valor aproximado es: 2.062500000058556
El error es: 5.855538276478e-11
El error relativo es: 2.8e-09 %
